In [1]:
import os
import shutil
from pydub import AudioSegment
import random
import pandas as pd 

c:\Users\Agata\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
csv_path = 'Sounds/dataset/Metadata_Train.csv'
audio_base_path = 'Sounds/dataset/Train_submission'  
output_base_path = 'Sounds/train' 

df = pd.read_csv(csv_path)

for _, row in df.iterrows():
    audio_file = row['FileName'] 
    audio_class = row['Class']  
    
    class_dir = os.path.join(output_base_path, str(audio_class))
    os.makedirs(class_dir, exist_ok=True)
    
    source_path = os.path.join(audio_base_path, audio_file)
    destination_path = os.path.join(class_dir, audio_file)
    
    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"Moved {audio_file} to {class_dir}")
    else:
        print(f"File {audio_file} not found in {audio_base_path}")

In [ ]:
audio_base_path = 'Sounds/train/Sound_Drum' 

shortest_audio = None
shortest_duration = float('inf')

for audio_file in os.listdir(audio_base_path):

    if audio_file.endswith('.wav'):
        file_path = os.path.join(audio_base_path, audio_file)
        audio = AudioSegment.from_file(file_path)
        duration = len(audio)
        
        if duration < shortest_duration:
            shortest_duration = duration
            shortest_audio = audio_file


shortest_duration_seconds = shortest_duration / 1000  

print(f"The shortest audio file is: {shortest_audio}")
print(f"It lasts: {shortest_duration_seconds:.2f} seconds")


In [ ]:
def create_segments(audio_base_path, output_base_path, segment_duration):
 
    os.makedirs(output_base_path, exist_ok=True)

    for audio_file in os.listdir(audio_base_path):
        if audio_file.endswith('.wav'):
            file_path = os.path.join(audio_base_path, audio_file)
            
            audio = AudioSegment.from_file(file_path)
            
            if len(audio) == segment_duration:
                print(f"File {audio_file} is already {segment_duration / 1000} seconds long. Skipping segmentation.")
                continue

            if len(audio) < segment_duration:
                print(f"Skipping {audio_file} as it is shorter than {segment_duration / 1000} seconds.")
                continue
            
            total_segments = len(audio) // segment_duration
            audio_name = os.path.splitext(audio_file)[0]

            for i in range(total_segments):
                start_time = i * segment_duration
                end_time = start_time + segment_duration
                segment = audio[start_time:end_time]
                
                segment_filename = f"{audio_name}_segment_{i+1}.wav"
                segment_path = os.path.join(output_base_path, segment_filename)
                
                segment.export(segment_path, format="wav")

def delete_short_segments(directory_path, min_duration):

    for audio_file in os.listdir(directory_path):
        if audio_file.endswith('.wav'):
            file_path = os.path.join(directory_path, audio_file)
            audio = AudioSegment.from_file(file_path)
            
            if len(audio) < min_duration:
                os.remove(file_path)


def delete_silent_segments(base_folder,silence_threshold):

    for root, dirs, files in os.walk(base_folder):
        for file in files:
        
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                
                segment = AudioSegment.from_file(file_path)
                loudness = segment.dBFS
                # print(f"{file_path}: {loudness:.2f} dBFS")

                if loudness < silence_threshold:
                    os.remove(file_path)


def count_files_in_folders(folders):

    for folder in folders:
        try:
            files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
            file_count = len(files)
            print(f"{folder}: {file_count} files")
        except FileNotFoundError:
            print(f"{folder}: Folder not found")


def move_files_between_folders(train_folder, test_folder, dataset_folder, instrument_folders):

    if not os.path.exists(dataset_folder):
        os.makedirs(dataset_folder)

    for subfolder in instrument_folders:
        train_subfolder = os.path.join(train_folder, subfolder)
        test_subfolder = os.path.join(test_folder, subfolder)
        dataset_subfolder = os.path.join(dataset_folder, subfolder)

        if not os.path.exists(dataset_subfolder):
            os.makedirs(dataset_subfolder)

        if os.path.exists(train_subfolder):
            for file_name in os.listdir(train_subfolder):
                if file_name.endswith(".wav"):
                    source_file = os.path.join(train_subfolder, file_name)
                    dest_file = os.path.join(dataset_subfolder, file_name)
                    shutil.move(source_file, dest_file)
                else:
                    print(f"Skipped {file_name} (not a .wav file)")
        else:
            print(f"Train subfolder {train_subfolder} does not exist")

        if os.path.exists(test_subfolder):
            for file_name in os.listdir(test_subfolder):
                if file_name.endswith(".wav"):
                    source_file = os.path.join(test_subfolder, file_name)
                    dest_file = os.path.join(dataset_subfolder, file_name)
                    shutil.move(source_file, dest_file)
                else:
                    print(f"Skipped {file_name} (not a .wav file)")
        else:
            print(f"Test subfolder {test_subfolder} does not exist")


def limit_files_in_folder(folder_path, limit):

    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if len(files) > limit:
        
        files_to_keep = set(random.sample(files, limit))
        for file in files:
            
            if file not in files_to_keep:
                os.remove(os.path.join(folder_path, file))
        print(f"Reduced '{folder_path}' to {limit} files.")
    else:
        print(f"'{folder_path}' already has {len(files)} files, which is within the limit.")

def split_files_into_train_and_test(source_folder, train_folder, test_folder, train_percentage=0.8):

    
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    for instrument_folder in os.listdir(source_folder):
        instrument_path = os.path.join(source_folder, instrument_folder)
        
        if not os.path.isdir(instrument_path):
            continue

        train_instrument_path = os.path.join(train_folder, instrument_folder)
        test_instrument_path = os.path.join(test_folder, instrument_folder)
        os.makedirs(train_instrument_path, exist_ok=True)
        os.makedirs(test_instrument_path, exist_ok=True)

        files = [f for f in os.listdir(instrument_path) if os.path.isfile(os.path.join(instrument_path, f))]
        total_files = len(files)

        num_train_files = int(total_files * train_percentage)
        train_files = random.sample(files, num_train_files)
        test_files = [f for f in files if f not in train_files]

        for file_name in train_files:
            source_file = os.path.join(instrument_path, file_name)
            destination_file = os.path.join(train_instrument_path, file_name)
            shutil.move(source_file, destination_file)
            # print(f"Moved {file_name} to {train_instrument_path}")

        for file_name in test_files:
            source_file = os.path.join(instrument_path, file_name)
            destination_file = os.path.join(test_instrument_path, file_name)
            shutil.move(source_file, destination_file)
            # print(f"Moved {file_name} to {test_instrument_path}")


# 1 sekundowe pliki

In [ ]:
audio_base_path = 'Sounds/train/Sound_Drum'
output_base_path = 'Sounds_1_seconds/train/Drum'

create_segments(audio_base_path, output_base_path, 1000)
delete_short_segments(output_base_path, 1000)

In [ ]:
base_folder = 'Sounds_1_seconds'
silence_threshold = -60.0
delete_silent_segments(base_folder,silence_threshold)

In [ ]:
sec1_folders = [
    "Sounds_1_seconds/test/Drum",
    "Sounds_1_seconds/test/Guitar",
    "Sounds_1_seconds/test/Piano",
    "Sounds_1_seconds/test/Violin",

    "Sounds_1_seconds/train/Drum",
    "Sounds_1_seconds/train/Guitar",
    "Sounds_1_seconds/train/Piano",
    "Sounds_1_seconds/train/Violin"
]

count_files_in_folders(sec1_folders)

In [ ]:
train_folder = "Sounds_1_seconds/train"
test_folder = "Sounds_1_seconds/test"
dataset_folder = "Sounds_1_seconds/sum"
instrument_folders = ["Drum", "Guitar", "Piano", "Violin"]

move_files_between_folders(train_folder, test_folder, dataset_folder, instrument_folders)

In [ ]:
sec1_folders = [
    "Sounds_1_seconds/sum/Drum",
    "Sounds_1_seconds/sum/Guitar",
    "Sounds_1_seconds/sum/Piano",
    "Sounds_1_seconds/sum/Violin"
]

for folder in sec1_folders:
    limit_files_in_folder(folder, 8300)

In [ ]:
source_folder = "Sounds_1_seconds/sum"
train_folder = "Sounds_1_seconds/train"
test_folder = "Sounds_1_seconds/test"

split_files_into_train_and_test(source_folder, train_folder, test_folder, train_percentage=0.8)

# 3 sekundowe pliki

In [12]:
audio_base_path = 'Sounds/test/Sound_Violin'
output_base_path = 'Sounds_3_seconds/test/Violin'

create_segments(audio_base_path, output_base_path, 3000)
delete_short_segments(output_base_path, 3000)

In [13]:
base_folder = 'Sounds_3_seconds'
silence_threshold = -60.0
delete_silent_segments(base_folder,silence_threshold)

In [14]:
sec3_folders = [
    "Sounds_3_seconds/test/Drum",
    "Sounds_3_seconds/test/Guitar",
    "Sounds_3_seconds/test/Piano",
    "Sounds_3_seconds/test/Violin",

    "Sounds_3_seconds/train/Drum",
    "Sounds_3_seconds/train/Guitar",
    "Sounds_3_seconds/train/Piano",
    "Sounds_3_seconds/train/Violin"
]

count_files_in_folders(sec3_folders)

Sounds_3_seconds/test/Drum: 134 files
Sounds_3_seconds/test/Guitar: 246 files
Sounds_3_seconds/test/Piano: 174 files
Sounds_3_seconds/test/Violin: 40 files
Sounds_3_seconds/train/Drum: 2832 files
Sounds_3_seconds/train/Guitar: 2869 files
Sounds_3_seconds/train/Piano: 5376 files
Sounds_3_seconds/train/Violin: 5526 files


In [15]:
train_folder = "Sounds_3_seconds/train"
test_folder = "Sounds_3_seconds/test"
dataset_folder = "Sounds_3_seconds/sum"
instrument_folders = ["Drum", "Guitar", "Piano", "Violin"]

move_files_between_folders(train_folder, test_folder, dataset_folder, instrument_folders)

In [16]:
# Drum - 2966 / Violin - 5566 / Piano - 5550 / Guitar - 3115

sec3_folders = [
    "Sounds_3_seconds/sum/Drum",
    "Sounds_3_seconds/sum/Guitar",
    "Sounds_3_seconds/sum/Piano",
    "Sounds_3_seconds/sum/Violin"
]

for folder in sec3_folders:
    limit_files_in_folder(folder, 2900)

Reduced 'Sounds_3_seconds/sum/Drum' to 2900 files.
Reduced 'Sounds_3_seconds/sum/Guitar' to 2900 files.
Reduced 'Sounds_3_seconds/sum/Piano' to 2900 files.
Reduced 'Sounds_3_seconds/sum/Violin' to 2900 files.


In [17]:
source_folder = "Sounds_3_seconds/sum"
train_folder = "Sounds_3_seconds/train"
test_folder = "Sounds_3_seconds/test"

split_files_into_train_and_test(source_folder, train_folder, test_folder, train_percentage = 0.8)

In [31]:
sec3_folders = [
    "Sounds_3_seconds/test/Drum",
    "Sounds_3_seconds/test/Guitar",
    "Sounds_3_seconds/test/Piano",
    "Sounds_3_seconds/test/Violin",

    "Sounds_3_seconds/train/Drum",
    "Sounds_3_seconds/train/Guitar",
    "Sounds_3_seconds/train/Piano",
    "Sounds_3_seconds/train/Violin"
]

count_files_in_folders(sec3_folders)

Sounds_3_seconds/test/Drum: 580 files
Sounds_3_seconds/test/Guitar: 580 files
Sounds_3_seconds/test/Piano: 580 files
Sounds_3_seconds/test/Violin: 580 files
Sounds_3_seconds/train/Drum: 2320 files
Sounds_3_seconds/train/Guitar: 2320 files
Sounds_3_seconds/train/Piano: 2320 files
Sounds_3_seconds/train/Violin: 2320 files


# 5 sekundowe pliki

In [25]:
audio_base_path = 'Sounds/train/Sound_Violin'
output_base_path = 'Sounds_5_seconds/train/Violin'

create_segments(audio_base_path, output_base_path, 5000)
delete_short_segments(output_base_path, 5000)

Skipping 14_ray-chen_bwv1004_mov5.wav as it is shorter than 5.0 seconds.
Skipping VIOLIN_SOUND (144).wav as it is shorter than 5.0 seconds.


In [26]:
base_folder = 'Sounds_5_seconds'
silence_threshold = -60.0
delete_silent_segments(base_folder,silence_threshold)

In [ ]:
sec5_folders = [
    "Sounds_5_seconds/test/Drum",
    "Sounds_5_seconds/test/Guitar",
    "Sounds_5_seconds/test/Piano",
    "Sounds_5_seconds/test/Violin",

    "Sounds_5_seconds/train/Drum",
    "Sounds_5_seconds/train/Guitar",
    "Sounds_5_seconds/train/Piano",
    "Sounds_5_seconds/train/Violin"
]

count_files_in_folders(sec5_folders)

5_seconds_sounds/test/Drum: 77 files
5_seconds_sounds/test/Guitar: 145 files
5_seconds_sounds/test/Piano: 100 files
5_seconds_sounds/test/Violin: 21 files
5_seconds_sounds/train/Drum: 1562 files
5_seconds_sounds/train/Guitar: 1668 files
5_seconds_sounds/train/Piano: 3188 files
5_seconds_sounds/train/Violin: 3224 files


In [27]:
train_folder = "Sounds_5_seconds/train"
test_folder = "Sounds_5_seconds/test"
dataset_folder = "Sounds_5_seconds/sum"
instrument_folders = ["Drum", "Guitar", "Piano", "Violin"]

move_files_between_folders(train_folder, test_folder, dataset_folder, instrument_folders)

In [28]:
# Drum - 1639 / Violin - 3245 / Piano - 3288 / Guitar - 1813

sec5_folders = [
    "Sounds_5_seconds/sum/Drum",
    "Sounds_5_seconds/sum/Guitar",
    "Sounds_5_seconds/sum/Piano",
    "Sounds_5_seconds/sum/Violin"
]

for folder in sec5_folders:
    limit_files_in_folder(folder, 1600)

Reduced 'Sounds_5_seconds/sum/Drum' to 1600 files.
Reduced 'Sounds_5_seconds/sum/Guitar' to 1600 files.
Reduced 'Sounds_5_seconds/sum/Piano' to 1600 files.
Reduced 'Sounds_5_seconds/sum/Violin' to 1600 files.


In [29]:
source_folder = "Sounds_5_seconds/sum"
train_folder = "Sounds_5_seconds/train"
test_folder = "Sounds_5_seconds/test"

split_files_into_train_and_test(source_folder, train_folder, test_folder, train_percentage = 0.8)

In [30]:
sec5_folders = [
    "Sounds_5_seconds/test/Drum",
    "Sounds_5_seconds/test/Guitar",
    "Sounds_5_seconds/test/Piano",
    "Sounds_5_seconds/test/Violin",

    "Sounds_5_seconds/train/Drum",
    "Sounds_5_seconds/train/Guitar",
    "Sounds_5_seconds/train/Piano",
    "Sounds_5_seconds/train/Violin"
]

count_files_in_folders(sec5_folders)

Sounds_5_seconds/test/Drum: 320 files
Sounds_5_seconds/test/Guitar: 320 files
Sounds_5_seconds/test/Piano: 320 files
Sounds_5_seconds/test/Violin: 320 files
Sounds_5_seconds/train/Drum: 1280 files
Sounds_5_seconds/train/Guitar: 1280 files
Sounds_5_seconds/train/Piano: 1280 files
Sounds_5_seconds/train/Violin: 1280 files
